In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.nn.functional as F
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader,Subset
from tqdm import tqdm
import pandas as pd
import time

In [2]:
# 데이터 전처리 (CIFAR10은 3채널 이미지)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])
])

# CIFAR10 학습 및 테스트 데이터셋 불러오기
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)

# CIFAR10의 targets는 리스트로 되어 있으므로 tensor로 변환
targets = torch.tensor(train_dataset.targets)

num_samples_per_class = 1700
selected_indices = []

# 0부터 9까지 각 클래스별로 인덱스를 추출하고, 랜덤하게 num_samples_per_class개 선택
for class_label in range(10):
    # 해당 클래스의 인덱스 추출
    indices = (targets == class_label).nonzero(as_tuple=True)[0]
    # 인덱스를 랜덤하게 섞은 후 원하는 개수만 선택
    selected = indices[torch.randperm(len(indices))[:num_samples_per_class]]
    selected_indices.extend(selected.tolist())

# 선택한 인덱스만 남긴 서브셋 생성
train_dataset = Subset(train_dataset, selected_indices)

100%|██████████| 170M/170M [00:02<00:00, 77.3MB/s]


In [3]:
import torch
import torch.nn as nn

class VGG11(nn.Module):
    def __init__(self):
        super(VGG11, self).__init__()
        self.features = nn.Sequential(
            # block 1
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # block 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # block 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(256 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 10),
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)


class VGG11_7x7(nn.Module):
    def __init__(self):
        super(VGG11_7x7, self).__init__()
        self.features = nn.Sequential(
            # block 1: 7x7 하나로 3x3 3개 대체
            nn.Conv2d(3, 64, kernel_size=7, padding=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # block 2
            nn.Conv2d(64, 128, kernel_size=7, padding=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # block 3
            nn.Conv2d(128, 256, kernel_size=7, padding=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.Linear(256 * 2 * 2, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 10),
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)


In [4]:
import time
import torch
import random
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm

def worker_init_fn(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def run_experiment(model_cls, seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=8, worker_init_fn=worker_init_fn)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False, num_workers=8, worker_init_fn=worker_init_fn)

    model = model_cls().to(device)
    params = count_parameters(model)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=4e-4)
    num_epochs = 30

    print(f"\n=== {model_cls.__name__} ===")
    print(f"Parameters: {params:,}")

    # 전체 훈련 시간 측정
    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=True)
        for data, target in progress_bar:
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})
        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}, lr = {optimizer.param_groups[0]['lr']}")
    end_time = time.time()

    total_train_time = end_time - start_time

    # 테스트 정확도 측정
    model.eval()
    correct = 0
    start_time = time.time()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            correct += (output.argmax(dim=1) == target).sum().item()

    test_acc = 100. * correct / len(test_loader.dataset)
    test_time = time.time() - start_time

    print(f"Total Train Time: {total_train_time:.1f}s")
    print(f"Total Test Time       : {test_time:.1f}s")
    print(f"Test Accuracy   : {test_acc:.2f}%")

    return {
        'model': model_cls.__name__,
        'params': params,
        'train_time': total_train_time,
        'test_time': test_time,
        'test_acc': test_acc
    }

In [6]:
if __name__ == "__main__":
    results = []
    num_experiments = 3

    for i in range(num_experiments):
        seed = np.random.randint(10000)
        result = []
        print(f"[Seed : {seed} Experiment Start]\n")
        result.append(run_experiment(VGG11_7x7, seed))
        result.append(run_experiment(VGG11, seed))
        results.append(result)

[Seed : 934 Experiment Start]


=== VGG11_7x7 ===
Parameters: 23,037,578


Epoch 1/30: 100%|██████████| 34/34 [00:05<00:00,  5.78it/s, Loss=1.7344]


Epoch 1, Loss: 1.9979, lr = 0.0004


Epoch 2/30: 100%|██████████| 34/34 [00:05<00:00,  5.88it/s, Loss=1.6484]


Epoch 2, Loss: 1.6035, lr = 0.0004


...

Epoch 29/30: 100%|██████████| 34/34 [00:06<00:00,  5.64it/s, Loss=0.1531]


Epoch 29, Loss: 0.1308, lr = 0.0004


Epoch 30/30: 100%|██████████| 34/34 [00:06<00:00,  5.65it/s, Loss=0.1033]

Epoch 30, Loss: 0.1125, lr = 0.0004


Total Train Time: 180.7s
Total Test Time       : 1.6s
Test Accuracy   : 67.53%


In [23]:
print("[Experiment Result]\n")
metrics = ["Parameters", "Parameters(Only Conv)", "Param Efficiency (% per 10M)", "Train Time (s)", "Test Time (s)", "Test Accuracy (%)"]
dfs = []

for i, result in enumerate(results):
    print(f"[Experiment {i+1}]")
    # FC 레이어 파라미터 수 = 21,020,682
    data = {
        result[0]['model']: [
            result[0]['params'],
            result[0]['params'] - 21020682,
            round(result[0]['test_acc'] / (result[0]['params'] / 1e7), 2),
            round(result[0]['train_time'], 1),
            round(result[0]['test_time'], 1),
            result[0]['test_acc']
        ],
        result[1]['model']: [
            result[1]['params'],
            result[1]['params'] - 21020682,
            round(result[1]['test_acc'] / (result[1]['params'] / 1e7), 2),
            round(result[1]['train_time'], 1),
            round(result[1]['test_time'], 1),
            result[1]['test_acc']
        ],
    }


    df = pd.DataFrame(data, index=metrics)
    df = df.astype(float)
    df["Difference (3x3 - 7x7)"] = df["VGG11"] - df["VGG11_7x7"]
    print(df, "\n")
    dfs.append(df)

print("[Average Result]\n")
avg_df = sum(dfs) / num_experiments
print(avg_df.round(2))

print("\n[Analysis]")
print(f"- VGG11_7x7 구조는 파라미터가 평균적으로 {abs(avg_df.loc['Parameters','Difference (3x3 - 7x7)']):.0f}개 적다.")
print(f"- 파라미터 효율 차이는 {avg_df.loc['Param Efficiency (% per 10M)','Difference (3x3 - 7x7)']:.2f} (% per M params)")
print(f"- 학습 시간 차이는 {avg_df.loc['Train Time (s)','Difference (3x3 - 7x7)']:.2f}s")
print(f"- 테스트 시간 차이는 {avg_df.loc['Test Time (s)','Difference (3x3 - 7x7)']:.2f}s")
print(f"- 테스트 정확도 차이는 {avg_df.loc['Test Accuracy (%)','Difference (3x3 - 7x7)']:.2f}%")
print(f"- 학습 시간 효율(비율)은 {avg_df.loc['Train Time (s)','VGG11'] / avg_df.loc['Train Time (s)','VGG11_7x7']:.2f}배")

[Experiment Result]

[Experiment 1]
                                VGG11_7x7        VGG11  Difference (3x3 - 7x7)
Parameters                    23037578.00  22940682.00               -96896.00
Parameters(Only Conv)          2016896.00   1920000.00               -96896.00
Param Efficiency (% per 10M)        27.68        28.88                    1.20
Train Time (s)                     173.50       180.70                    7.20
Test Time (s)                        1.60         1.50                   -0.10
Test Accuracy (%)                   63.77        66.25                    2.48 

[Experiment 2]
                                VGG11_7x7        VGG11  Difference (3x3 - 7x7)
Parameters                    23037578.00  22940682.00               -96896.00
Parameters(Only Conv)          2016896.00   1920000.00               -96896.00
Param Efficiency (% per 10M)        28.13        29.17                    1.04
Train Time (s)                     174.60       180.90                    6.30

=> 기존 VGGNet(3x3 Conv×3) 방식이 7x7 Conv 대체 모델보다 모든 지표에서 우세하였다.
 구체적으로, VGGNet은 7x7로 대체한 네트워크에 비해 적은 파라미터에도 평균 정확도가 약 3%p 높고, 레이어 깊이 증가로 인해 늘어난 학습 시간에도, 파라미터 효율 및 학습 시간 대비 정확도(accuracy/time)가 일관되게 앞섰다,
 Test Time에는 큰 차이가 없는 것으로 미루어, 3x3 conv만을 사용하더라도 추가적인 시간 병목 없이 Inference를 수행할 수 있을 것이다.
 이를 통해 '깊은 3x3 합성곱 + 비선형성의 누적'을 통해 7x7 Conv 하나로는 대체할 수 없는 성능 이점을 제공함을 확인할 수 있다.